# Tuning the model


## Load the data
Load the preprocessed data which prepared in HDF5 format. The data is split into training, validation, and test sets. The training set is used to train the model, the validation set is used to tune the hyperparameters, and the test set is used to evaluate the final model performance.

In [ ]:
from dataset import CHBMITDataset

data_path = "./CHB-MIT/processed"
dataset = CHBMITDataset(data_path)

## Assemble the model

We will use the Pytorch Lightning library to build the model pipeline from the based model.

In [ ]:
from dataset.lightning import CHBMITDataModule

datamodule = CHBMITDataModule(dataset, batch_size=32)

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback

from encoder import PoissonEncoder
from model.classifier import ModelConfig
from model.lightning import LitSTFTSeizureClassifier, OptimizerConfig


def objective(trial: optuna.trial.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.1),
        "slope": trial.suggest_float("slope", 5.0, 20.0),
        "beta": trial.suggest_float("beta", 0.8, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.3, 0.7),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.4),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-5, 1e-2, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.5),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 3, 10),
    }

    spike_encoder = PoissonEncoder()

    model = LitSTFTSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=15,
        accelerator="cuda",
        devices=1,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")],
    )

    trainer.fit(model, datamodule=datamodule)

    return trainer.callback_metrics["val_loss"].item()



In [ ]:
from config import DB_CONFIG

study_name = "Classifier Rate Encoder"
storage_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

study = optuna.create_study(
    direction="minimize",
    study_name=study_name,
    storage=storage_url,
    load_if_exists=True,
)

[I 2025-04-01 23:00:47,898] Using an existing study with name 'Classifier Rate Encoder' instead of creating a new one.


In [14]:
study.optimize(objective, n_trials=15)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type               | Params | Mode 
-----------------------------------------------------
0 | model | EEGSpikeClassifier | 824 K  | train
-----------------------------------------------------
824 K     Trainable params
0         Non-trainable params
824 K     Total params
3.299     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2025-04-01 23:01:57,828] Trial 8 failed with parameters: {'threshold': 0.08013968821261705, 'slope': 7.492422674915332, 'beta': 0.8621571437924727, 'dropout_rate1': 0.681498163181113, 'dropout_rate2': 0.11504787198225014, 'lr': 0.0005640386380741711, 'weight_decay': 0.006364326224720527, 'scheduler_factor': 0.1991904968995094, 'scheduler_patience': 9} because of the following error: NameError("name 'exit' is not defined").
Traceback (most recent call last):
  File "e:\Projects\snn-encoder-test\.venv\Lib\site-packages\lightning\pytorch\trainer\call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Projects\snn-encoder-test\.venv\Lib\site-packages\lightning\pytorch\trainer\trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "e:\Projects\snn-encoder-test\.venv\Lib\site-packages\lightning\pytorch\trainer\trainer

NameError: name 'exit' is not defined

In [ ]:
study.best_params